In [1]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pymupdf
!pip install langchain langchain-core langchain-community langchain-huggingface langchain-openai faiss-cpu

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.38
    Uninstalling langchain-core-0.2.38:
      Successfully uninstalled langchain-core-0.2.38
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [3]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached langchain_core-0.2.38-py3-none-any.whl.metadata (6.2 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
Using cached langchain_core-0.2.38-py3-none-any.whl (396 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.2
    Uninstalling sentence-transformers-5.1.2:
      Successfully uninstalled sentence-transformers-5.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.2.2 which is incompatible.
langchain-openai 0

In [13]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

CUDA available: True
GPU Name: NVIDIA GeForce RTX 5060 Laptop GPU


In [5]:
!pip install langchain_community

In [6]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install sentence-transformers

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.chains import PebbloRetrievalQA
import os

In [21]:
docs = []
pdf_path = os.path.join("data","userguide.pdf")
pdf_loader = PyMuPDFLoader("userguide.pdf")
print(pdf_loader)
docs.extend(pdf_loader.load())
md_path = "CFM_OAM.md"
md_loader = TextLoader(md_path, encoding='utf-8')
docs.extend(md_loader.load())
print(f"Loaded {len(docs)} documents")

Loaded 1069 documents


In [23]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device}")

🚀 Using device: cuda


In [11]:
# #ignore this
# #
# #

# import logging
# import shutil
# import gc
# from langchain_huggingface import HuggingFaceEmbeddings
# import chromadb
# import pickle

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

# # Define constants
# VECTOR_DB_DIR = "embeddings"
# CHUNK_SIZE = 300  # Reduced from 500 to create fewer, smaller chunks
# CHUNK_OVERLAP = 50  # Reduced from 100
# BATCH_SIZE = 10  # Reduced from 50
# MODEL_NAME = "all-MiniLM-L6-v2"
# EMBEDDINGS_FILE = "embeddings.pkl"
# IDS_FILE = "ids.pkl"
# METADATAS_FILE = "metadatas.pkl"

# # Step 1: Delete existing Chroma database
# if os.path.exists("chroma_db"):
#     try:
#         shutil.rmtree("chroma_db")
#         logger.info("Existing Chroma database deleted")
#     except Exception as e:
#         logger.error(f"Failed to delete chroma_db: {e}")
#         raise

# # Step 2: Split documents into chunks
# try:
#     splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
#     chunks = splitter.split_documents(docs)
#     logger.info(f"Split into {len(chunks)} chunks")
# except Exception as e:
#     logger.error(f"Error during document splitting: {e}")
#     raise

# # Step 3: Initialize embedding function
# try:
#     embedding_function = HuggingFaceEmbeddings(
#         model_name=MODEL_NAME,
#         model_kwargs={"device": "cuda"},
#         encode_kwargs={"batch_size": 4}  # Reduced from 8
#     )
#     logger.info("Embedding function initialized")
# except Exception as e:
#     logger.error(f"Error initializing embedding function: {e}")
#     raise

# # Step 4: Precompute embeddings to avoid in-memory Chroma issues
# if not os.path.exists(EMBEDDINGS_FILE):
#     try:
#         logger.info("Precomputing embeddings...")
#         chunk_texts = [chunk.page_content for chunk in chunks]
#         metadatas = [chunk.metadata for chunk in chunks]
#         ids = [f"chunk_{i}" for i in range(len(chunks))]
#         embeddings = []
#         for i in range(0, len(chunk_texts), 4):
#             batch_texts = chunk_texts[i:i + 4]
#             batch_embeddings = embedding_function.embed_documents(batch_texts)
#             embeddings.extend(batch_embeddings)
#             logger.info(f"Embedded batch {i // 4 + 1} of {len(chunk_texts) // 4 + 1}")
#             gc.collect()
        
#         # Save embeddings, IDs, and metadatas
#         with open(EMBEDDINGS_FILE, "wb") as f:
#             pickle.dump(embeddings, f)
#         with open(IDS_FILE, "wb") as f:
#             pickle.dump(ids, f)
#         with open(METADATAS_FILE, "wb") as f:
#             pickle.dump(metadatas, f)
#         logger.info("Embeddings saved to disk")
#     except Exception as e:
#         logger.error(f"Error precomputing embeddings: {e}")
#         raise
# else:
#     logger.info("Loading precomputed embeddings...")
#     with open(EMBEDDINGS_FILE, "rb") as f:
#         embeddings = pickle.load(f)
#     with open(IDS_FILE, "rb") as f:
#         ids = pickle.load(f)
#     with open(METADATAS_FILE, "rb") as f:
#         metadatas = pickle.load(f)
#     logger.info("Precomputed embeddings loaded")

# # Step 5: Initialize Chroma client
# try:
#     client = chromadb.PersistentClient(path="chroma_db")
#     logger.info("Chroma client initialized")
# except Exception as e:
#     logger.error(f"Error initializing Chroma client: {e}")
#     raise

# # Step 6: Add embeddings to Chroma in batches
# try:
#     collection = client.get_or_create_collection(
#         name="user_docs",
#         metadata={"hnsw:space": "cosine"}
#     )
#     for i in range(0, len(chunks), BATCH_SIZE):
#         batch_embeddings = embeddings[i:i + BATCH_SIZE]
#         batch_ids = ids[i:i + BATCH_SIZE]
#         batch_metadatas = metadatas[i:i + BATCH_SIZE]
#         batch_texts = [chunk.page_content for chunk in chunks[i:i + BATCH_SIZE]]
        
#         logger.info(f"Adding batch {i // BATCH_SIZE + 1} with {len(batch_embeddings)} documents to Chroma")
#         collection.add(
#             embeddings=batch_embeddings,
#             documents=batch_texts,
#             metadatas=batch_metadatas,
#             ids=batch_ids
#         )
#         logger.info(f"Batch {i // BATCH_SIZE + 1} added")
#         gc.collect()
# except Exception as e:
#     logger.error(f"Error adding batch to Chroma: {e}")
#     raise

# logger.info("✅ Chroma vector store created!")

In [24]:
import logging
import shutil
import gc
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
import pickle
import gc
import pickle
import numpy as np
from faiss import IndexFlatL2, write_index, read_index
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

EMBEDDINGS_FILE = "embeddings.pkl"
IDS_FILE = "ids.pkl"
METADATAS_FILE = "metadatas.pkl"
FAISS_INDEX_FILE = "faiss_index.bin"
DOCUMENTS_FILE = "documents.pkl"
# Define constants
VECTOR_DB_DIR = "embeddings"
CHUNK_SIZE = 500  # Reduced from 500 to create fewer, smaller chunks
CHUNK_OVERLAP = 50  # Reduced from 100
BATCH_SIZE = 10  # Reduced from 50
MODEL_NAME = "all-MiniLM-L6-v2"
EMBEDDINGS_FILE = "embeddings.pkl"
IDS_FILE = "ids.pkl"
METADATAS_FILE = "metadatas.pkl"

# Step 2: Split documents into chunks
try:
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    logger.info(f"Split into {len(chunks)} chunks")
except Exception as e:
    logger.error(f"Error during document splitting: {e}")
    raise

# Step 3: Precompute embeddings
try:
    logger.info("Initializing embedding function...")
    embedding_function = HuggingFaceEmbeddings(
        model_name=MODEL_NAME,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"batch_size": BATCH_SIZE}
    )
    logger.info("Embedding function initialized")
    
    logger.info("Precomputing embeddings...")
    chunk_texts = [chunk.page_content for chunk in chunks]
    metadatas = [chunk.metadata for chunk in chunks]
    ids = [f"chunk_{i}" for i in range(len(chunks))]
    embeddings = []
    for i in range(0, len(chunk_texts), BATCH_SIZE):
        batch_texts = chunk_texts[i:i + BATCH_SIZE]
        batch_embeddings = embedding_function.embed_documents(batch_texts)
        embeddings.extend(batch_embeddings)
        logger.info(f"Embedded batch {i // BATCH_SIZE + 1} of {len(chunk_texts) // BATCH_SIZE + 1}")
        gc.collect()
    
    # Save embeddings, IDs, metadatas, and documents
    with open(EMBEDDINGS_FILE, "wb") as f:
        pickle.dump(embeddings, f)
    with open(IDS_FILE, "wb") as f:
        pickle.dump(ids, f)
    with open(METADATAS_FILE, "wb") as f:
        pickle.dump(metadatas, f)
    with open(DOCUMENTS_FILE, "wb") as f:
        pickle.dump(chunk_texts, f)
    logger.info("Embeddings and documents saved to disk")
except Exception as e:
    logger.error(f"Error precomputing embeddings: {e}")
    raise

# Create FAISS index (FlatL2 for exact search; use IndexHNSWFlat for cosine)
try:
    embeddings_np = np.array(embeddings, dtype=np.float32)
    index = IndexFlatL2(embeddings_np.shape[1])  # L2 distance (can use cosine with normalization)
    index.add(embeddings_np)
    write_index(index, FAISS_INDEX_FILE)
    logger.info("FAISS index created and saved")
except Exception as e:
    logger.error(f"Error creating FAISS index: {e}")
    raise

# Save documents for retrieval
with open(DOCUMENTS_FILE, "wb") as f:
    pickle.dump([chunk.page_content for chunk in chunks], f)

try:
    index = read_index(FAISS_INDEX_FILE)
    if index.ntotal == 0:
        raise ValueError("FAISS index is empty")
    logger.info(f"FAISS index contains {index.ntotal} vectors")
except Exception as e:
    logger.error(f"Error validating FAISS index: {e}")
    raise

2025-10-29 07:25:39,665 - INFO - Split into 3502 chunks
2025-10-29 07:25:39,666 - INFO - Initializing embedding function...
2025-10-29 07:25:39,674 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-10-29 07:25:42,514 - INFO - Embedding function initialized
2025-10-29 07:25:42,515 - INFO - Precomputing embeddings...
2025-10-29 07:25:43,349 - INFO - Embedded batch 1 of 351
2025-10-29 07:25:43,748 - INFO - Embedded batch 2 of 351
2025-10-29 07:25:44,276 - INFO - Embedded batch 3 of 351
2025-10-29 07:25:44,831 - INFO - Embedded batch 4 of 351
2025-10-29 07:25:45,332 - INFO - Embedded batch 5 of 351
2025-10-29 07:25:45,814 - INFO - Embedded batch 6 of 351
2025-10-29 07:25:46,217 - INFO - Embedded batch 7 of 351
2025-10-29 07:25:46,605 - INFO - Embedded batch 8 of 351
2025-10-29 07:25:47,005 - INFO - Embedded batch 9 of 351
2025-10-29 07:25:47,417 - INFO - Embedded batch 10 of 351
2025-10-29 07:25:47,860 - INFO - Embedded batch 11 of 351
2025-10-29 07:25:48,297 - INFO - Emb

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "all-MiniLM-L6-v2"

# Step 6: Query FAISS for RAG
try:
    query = "I need Lightspan RPC to initiate Linktrace test"
    query_embedding = np.array(embedding_function.embed_query(query), dtype=np.float32)
    k = min(5, index.ntotal)  # Ensure k doesn’t exceed number of documents
    distances, indices = index.search(query_embedding.reshape(1, -1), k=k)
    logger.info(f"Retrieved {len(indices[0])} documents for query")
    
    # Create context, handling empty or invalid indices
    context = ""
    if len(indices[0]) > 0:
        context = "\n".join([f"Document {i+1}: {chunk_texts[idx]}" for i, idx in enumerate(indices[0]) if idx < len(chunk_texts)])
    else:
        logger.warning("No documents retrieved from FAISS")
        context = "No relevant documents found."
except Exception as e:
    logger.error(f"Error during FAISS query: {e}")
    raise

# Step 7: Initialize Qwen and generate response
try:
    tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-4B",
    trust_remote_code=True,
    use_fast=False
)

    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B").to(DEVICE)
    logger.info("Qwen model and tokenizer initialized")
    
    prompt = f"""Use the following context to answer the query:

    **Context**:
    {context}

    **Query**:
    {query}

    **Answer**: """
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(DEVICE)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    logger.info("Response generated")
    print("Response:", response)
except Exception as e:
    logger.error(f"Error generating response: {e}")
    raise


2025-10-29 07:33:39,140 - INFO - Retrieved 5 documents for query


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

2025-10-29 07:33:44,711 - ERROR - Error generating response: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.


ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [26]:
%pip install -U transformers tokenizers accelerate huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 1.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 430.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
Note: you may need to restart the kernel to use updated packages.


In [65]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ✅ FIX: disable fast tokenizer and trust remote code
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    trust_remote_code=True,
    use_fast=False  # 👈 this avoids the ModelWrapper JSON error
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
).to(device)

# Example context (replace with your RAG context)
context = "Lightspan RPC is used to initiate diagnostics and link trace operations on network elements."

user_query = "Give me the Lightspan RPC to initiate Loopback Series test"

# Build RAG-style prompt
prompt = f"""
You are a helpful assistant with access to documentation.

Use the following context to answer the question clearly and concisely.

Context:
{context}

Question:
{user_query}

Answer:
"""

messages = [
    {"role": "system", "content": "You are a knowledgeable assistant who answers based on provided documentation."},
    {"role": "user", "content": prompt},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,
        top_p=0.9
    )

response = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
)

print("\n💬 Model response:\n", response)


Using device: cuda


ImportError: cannot import name 'is_flash_attn_3_available' from 'transformers.utils' (/opt/conda/lib/python3.10/site-packages/transformers/utils/__init__.py)

In [53]:
!pip install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu121
!pip install bitsandbytes accelerate transformers sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.9/767.9 MB 1.7 MB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 4.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 3.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 3.1 MB/s eta 0:00:0000:0100:01


In [63]:
!pip install -U "transformers>=4.45.0" "tokenizers>=0.20.0" "accelerate>=0.33.0" huggingface_hub safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
